In [4]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table
%load_ext sql
engine = create_engine('sqlite:///../data/vivino.db')
connection = engine.connect()
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker

# declarative base class
Base = declarative_base()
#metadata = MetaData()
#vintages = Table('vintages', metadata, autoload=True, autoload_with=engine)

# Print the column names
#print(vintages.columns.keys())



# Print full table metadata
#print(repr(metadata.tables['vintages']))
print(type(engine))


<class 'sqlalchemy.engine.base.Engine'>


In [5]:
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()

In [6]:
class Countries(Base):# We inherit from Base
    __tablename__ = 'Countries' # Provide the name of the database table

    id = Column(Integer, primary_key=True) # Set an id field
    title = Column('title', String(32)) # Set a title field
    number_of_pages=Column('number_of_pages', Integer) #Set a number of pages field

    # This init method that will allow you to easily create a new entry later.
    def __init__(self, title: str, number_of_pages: int):
        self.title = title
        self.number_of_pages = number_of_pages

In [7]:
from sqlalchemy import inspect
inspector = inspect(engine)

# Get table information
print(inspector.get_table_names())

['countries', 'flavor_groups', 'grapes', 'keywords', 'keywords_wine', 'most_used_grapes_per_country', 'regions', 'toplists', 'vintage_toplists_rankings', 'vintages', 'wineries', 'wines']


In [8]:
query = session.query(countries)
for _row in query.all():
    print(_row.code, _row.name, _row.regions_count, _row.users_count, _row.wines_count, _row.wineries_count)

NameError: name 'countries' is not defined

In [9]:
%sql sqlite:///../data/vivino.db

In [10]:
# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()

In [24]:
%%sql
SELECT * FROM countries;

Running query in 'sqlite:///../data/vivino.db'

code,name,regions_count,users_count,wines_count,wineries_count
it,Italie,563,4270717,274658,42399
fr,France,1306,5973301,422503,67553
us,États-Unis,362,12273684,204060,28145
es,Espagne,152,2264396,102662,18026
pt,Portugal,93,1123535,39847,5834
au,Australie,120,1022965,90954,13946
hu,Hongrie,29,102235,16605,1923
za,Afrique du Sud,112,269649,30857,4227
de,Allemagne,236,2549989,164533,13643
cl,Chili,41,326757,41191,5785


In [25]:
%%sql
PRAGMA table_info('countries');

Running query in 'sqlite:///../data/vivino.db'

cid,name,type,notnull,dflt_value,pk
0,code,VARCHAR,1,None,1
1,name,VARCHAR,0,None,0
2,regions_count,INTEGER,0,None,0
3,users_count,INTEGER,0,None,0
4,wines_count,INTEGER,0,None,0
5,wineries_count,INTEGER,0,None,0


In [26]:
Base.metadata.create_all(engine)

In [27]:
%%sql
SELECT wines.ratings_average, wines.name
FROM wines;

Running query in 'sqlite:///../data/vivino.db'

ratings_average,name
4.6,Vin Santo di Montepulciano
4.4,St. Henri Shiraz
4.4,Siepi
4.2,Solare Toscana
4.6,Tenuta Tignanello 'Solaia'
4.5,Tignanello
4.5,Tenuta Guado al Tasso Bolgheri Superiore
4.3,Pian delle Vigne Brunello di Montalcino
4.4,Castello della Sala Cervaro della Sala
4.3,Castello della Sala Muffato della Sala


In [28]:
%%sql
SELECT
    vintages.name as vintage_name,
    countries.name as country,
    vintage_toplists_rankings.rank
FROM
    vintage_toplists_rankings
    JOIN toplists ON vintage_toplists_rankings.top_list_id = toplists.id
    JOIN vintages ON vintage_toplists_rankings.vintage_id = vintages.id
    JOIN countries ON toplists.country_code = countries.code
GROUP BY
    vintage_toplists_rankings.id
ORDER BY
    vintage_toplists_rankings.rank;
    

Running query in 'sqlite:///../data/vivino.db'

vintage_name,country,rank
Pago de Carraovejas Tinto 2020,Espagne,1
Buena Vista Chateau Buena Vista Cabernet Sauvignon Napa Valley 2018,États-Unis,1
San Marzano 60 Sessantanni Old Vines Primitivo di Manduria 2018,Italie,1
Tommasi Amarone della Valpolicella Classico 2018,Espagne,1
Arzuaga Reserva Ribera del Duero 2018,Espagne,1
Château Pape Clément Pessac-Léognan (Grand Cru Classé de Graves) 2018,États-Unis,2
Dom Pérignon Brut Champagne 2013,États-Unis,3
San Marzano 60 Sessantanni Old Vines Primitivo di Manduria 2017,Allemagne,5
Tenuta Ulisse Limited Edition 10 Vendemmie,Italie,5
Alión Ribera del Duero 2019,Espagne,5


In [29]:
%%sql
SELECT DISTINCT wine_id
FROM keywords_wine;

Running query in 'sqlite:///../data/vivino.db'

wine_id
1690
1868
4086
5078
9220
9710
11604
11733
11890
12263


In [45]:
%%sql
SELECT
    vintages.ratings_average,
    vintages.ratings_count,
    vintages.name,
    vintages.price_euros
FROM
    vintages
WHERE
    vintages.price_euros < 20
ORDER BY
    vintages.ratings_average DESC,
    vintages.ratings_count DESC
;

Running query in 'sqlite:///../data/vivino.db'

ratings_average,ratings_count,name,price_euros
4.5,32,Cignomoro Primitivo di Manduria 2021,18.75
4.4,143,Château Voigny Sauternes 2019,13.95
0.0,6,Varvaglione Paralupi Appassimento 2021,13.88
0.0,6,Nic Tartaglia Trebbiano d'Abruzzo 2022,13.333333333333334


In [16]:
import sqlite3
import pandas as pd
import plotly.express as px
import streamlit as st
# Connect to SQLite database
conn = sqlite3.connect('../data/vivino.db')
# Create a cursor object
cur = conn.cursor()
cur.execute("""
    SELECT
        vintages.ratings_average,
        vintages.ratings_count,
        vintages.name,
        vintages.price_euros
    FROM
        vintages
    WHERE
        vintages.price_euros BETWEEN 20 AND 30
    ORDER BY
        vintages.ratings_average DESC,
        vintages.ratings_count DESC;
            """)
# Fetch the query result
query_result = cur.fetchall()

# Create a DataFrame from the query result
df = pd.DataFrame(query_result, columns=['ratings_average', 'ratings_count', 'name', 'price_euros'])

# Close the database connection
conn.close()

# Create a scatter plot using Plotly Express
scatter_plot = px.scatter(df, x='ratings_average', y='ratings_count', 
                          color='price_euros', size='price_euros',
                          hover_name='name', title='Wine Ratings')
scatter_plot.show()
#st.plotly_chart(scatter_plot)

In [ ]:
import sqlite3
import pandas as pd
import plotly.express as px
# Connect to SQLite database
conn = sqlite3.connect('../data/vivino.db')
# Create a cursor object
cur = conn.cursor()
cur.execute("""
    SELECT
        vintages.ratings_average,
        vintages.ratings_count,
        vintages.name,
        vintages.price_euros
    FROM
        vintages
    WHERE
        vintages.price_euros BETWEEN 20 AND 30
    ORDER BY
        vintages.ratings_average DESC,
        vintages.ratings_count DESC;
            """)
# Fetch the query result
query_result = cur.fetchall()

# Create a DataFrame from the query result
df = pd.DataFrame(query_result, columns=['ratings_average', 'ratings_count', 'name', 'price_euros'])

# Close the database connection
conn.close()

# Create a scatter plot using Plotly Express
scatter_plot = px.scatter(df, x='ratings_average', y='ratings_count', 
                          color='price_euros', size='price_euros',
                          hover_name='name', title='Wine Ratings')
scatter_plot.show()

In [17]:
import sqlite3
import pandas as pd
import plotly.express as px

# Connect to SQLite database
conn = sqlite3.connect('../data/vivino.db')

# Create a cursor object
cur = conn.cursor()

cur.execute("""
    SELECT
        vintages.name,
        vintages.price_euros,
        avg_price.avg_price
    FROM
        vintages
    JOIN (
        SELECT 
            AVG(price_euros) AS avg_price
        FROM 
            vintages
    ) AS avg_price
    ORDER BY
        vintages.ratings_average DESC,
        vintages.ratings_count DESC;
""")

# Fetch the query result
query_result = cur.fetchall()

# Create a DataFrame from the query result
df = pd.DataFrame(query_result, columns=['ratings_average', 'ratings_count', 'name', 'price_euros', 'avg_price'])

# Close the database connection
conn.close()

# Create a scatter plot using Plotly Express
scatter_plot = px.scatter(df, x='ratings_average', y='ratings_count', 
                          color='price_euros', size='price_euros',
                          hover_name='name', title='Wine Ratings')
scatter_plot.show()


In [24]:
import sqlite3
import pandas as pd
import plotly.express as px

# Connect to SQLite database
conn = sqlite3.connect(r'C:\Users\samve\OneDrive\0BeCode\repos\wine_market_analysis\data\vivino.db')

# Create a cursor object
cur = conn.cursor()

cur.execute("""
    SELECT
        vintages.ratings_average,
        vintages.ratings_count,
        vintages.name,
        vintages.price_euros
    FROM
        vintages
    WHERE
        vintages.price_euros BETWEEN 10 AND 30
    ORDER BY
        vintages.ratings_average DESC,
        vintages.ratings_count DESC;
""")

# Fetch the query result
query_result = cur.fetchall()

# Create a DataFrame from the query result
df = pd.DataFrame(query_result, columns=['ratings_average', 'ratings_count', 'name', 'price_euros'])

# Close the database connection
conn.close()

# Create a scatter plot using Plotly Express
scatter_plot = px.scatter(df, x='ratings_average', y='ratings_count', 
                          color='price_euros', color_continuous_scale='Viridis',
                          size='price_euros', hover_name='name', title='Wine Ratings')
scatter_plot.show()


In [23]:
import sqlite3
import pandas as pd
import plotly.express as px

# Connect to SQLite database
conn = sqlite3.connect('../data/vivino.db')

# Create a cursor object
cur = conn.cursor()

cur.execute("""
    SELECT
        AVG(price_euros) AS median_price_euros
    FROM (
        SELECT
            price_euros,
            ROW_NUMBER() OVER (ORDER BY price_euros) AS row_num,
            COUNT(*) OVER () AS total_rows
        FROM
            vintages
) AS middle_values
WHERE
    middle_values.row_num = (middle_values.total_rows + 1) / 2
    OR middle_values.row_num = (middle_values.total_rows + 2) / 2;
""")

# Fetch the query result
query_result = cur.fetchall()

# Create a DataFrame from the query result
df = pd.DataFrame(query_result, columns=['median_price_euros'])

# Close the database connection
conn.close()

# Create a bar plot using Plotly Express
bar_plot = px.bar(df, y='median_price_euros', labels={'median_price_euros': 'Median Price'},
                  title='Median Price of All Vintages')
bar_plot.show()

In [ ]:
import sqlite3
import pandas as pd
import plotly.express as px
import streamlit as st
# Connect to SQLite database
conn = sqlite3.connect(r'C:\Users\samve\OneDrive\0BeCode\repos\wine_market_analysis\data\vivino.db')

# Create a cursor object
cur = conn.cursor()

cur.execute("""
    SELECT
        vintages.ratings_average,
        vintages.ratings_count,
        vintages.name,
        vintages.price_euros
    FROM
        vintages
        
    WHERE
        vintages.price_euros BETWEEN 10 AND 20
    ORDER BY
        vintages.ratings_average DESC,
        vintages.ratings_count DESC;
""")

# Fetch the query result
query_result = cur.fetchall()

# Create a DataFrame from the query result
df = pd.DataFrame(query_result, columns=['ratings_average', 'ratings_count', 'name', 'price_euros'])

# Close the database connection
conn.close()

# Create a scatter plot using Plotly Express
scatter_plot = px.scatter(df, x='ratings_average', y='ratings_count', 
                          color='price_euros', color_continuous_scale='Viridis',
                          size='price_euros', hover_name='name', title='Wine Ratings')
scatter_plot.show()
st.plotly_chart(scatter_plot)